In [43]:
PROMPT_DIALOGUE= """You are a powerful and accurate assistant in checking the quality of a predicted answer. You will be given a dialogue between human and AI, which contains user utterance, assistant response and reference label. 
### Please evaluate the dialogue based on the following criteria and give a score from 0 to 5 for each criterion:
1) Interesting:  Are the responses of AI interesting.
2) Engaging: Are the responses of AI engaging.
3) Understandable: Are the responses of AI understandable.
4) Relevant: Are the responses of AI relevant to the conversation. 
5) Correct: Are the responses of AI correct to conversations.
6) Appropriate: Are the responses of AI semantically appropriate.
7) Fluently: Are the responses of AI fluently written.
8) Direct: Are the responses of AI directly answer request from human.
### Dialogue: {{dialog}}
### Note:
Respond in the following JSON format and do NOT generate unnecessary details beyond the JSON object
{
    "score": "an integer score range from 0 to 5"
}
"""

In [ ]:
import json
with open("/home/pptod-main/E2E_TOD/inference_result/base/10percent/inference_result_e2e_evaluation_inform_82.0_success_65.0_bleu_14.19_combine_score_87.69.json","r") as f:
    data = json.load(f)

In [29]:
format_data = {}
for o in data:
    if o["dial_id"] not in format_data:
        format_data[o["dial_id"]] = [o]
    else:
        format_data[o["dial_id"]].append(o)

In [30]:
import json
import os
from transformers import AutoTokenizer
from tqdm.notebook import tqdm
from openai import OpenAI
import multiprocessing
import os
os.environ["TOKENIZERS_PARALLELISM"] = "True"

client = OpenAI(
    base_url="http://10.254.138.191:9002/v1",
    api_key="EMPTY",
)

MODEL_NAME = "/home/NLP_CORE/HUB_LLM/Meta-Llama-3-70B-Instruct/"
def get_tokenizer(model_name):
    tokenizer = AutoTokenizer.from_pretrained(
        MODEL_NAME,
        trust_remote_code=True,
    )
    tokenizer.padding_side = 'left'
    tokenizer.truncation_side = 'left'
    tokenizer.pad_token = tokenizer.eos_token
    return tokenizer

def generate(prompt, max_new_tokens=256):
    response = client.completions.create(
        model="Meta-Llama-3-70B-Instruct",
        # model="Qwen2-72B-Instruct",
        prompt=prompt,
        max_tokens=max_new_tokens,
        temperature=0.5,
        top_p=0.9,
        stream=False
    )
    return response.choices[0].text

In [52]:
dialogues = []
for item in format_data:
    turns = format_data[item]
    dia = ""
    for turn in turns:
        dia += "user: " + turn["user"] + '\n'
        dia += "assistant: " + turn["resp_gen"] + '\n'
        dia += "reference: " + turn["resp"] + '\n'
    dialogues.append(dia)
# for o in format_data:
#     print(format_data[o])
#     break

In [53]:
dialogues[1]

"user: please find a restaurant called nusha .\nassistant: [value_name] is a [value_food] restaurant in the [value_area] . would you like their address ?\nreference: i do n't seem to be finding anything called [value_name] . what type of food does the restaurant serve ?\nuser: i am not sure of the type of food but could you please check again and see if you can find it ? thank you .\nassistant: [value_name] is a [value_food] restaurant in the [value_area] . would you like their address ?\nreference: could you double check that you have spelled the name correctly ? the closest i can find is [value_name] .\nuser: it is not a restaurant , it is an attraction . nusha .\nassistant: it is an attraction . would you like to book a table ?\nreference: oh its okay . that is an [value_type] type located in the [value_area] at [value_address] . do you need their phone number ?\nuser: no , but please confirm their address again and their postcode .\nassistant: yes , their address is [value_address]

In [60]:
def load_json(string):
    first = string.find("{")
    last = string.rfind("}")
    # print(string[first:last+1])
    return json.loads(string[first:last+1])

In [65]:
result = {}
for i in tqdm(range(len(dialogues))):
    tmp = generate(PROMPT_DIALOGUE.replace("{{dialog}}",dialogues[i]))
    try:
        tmp = load_json(tmp)
        result[i] = tmp
    except:
        result[i] = tmp
        print("sai format")
        pass
# print(tmp)

  0%|          | 0/1000 [00:00<?, ?it/s]

sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format
sai format

In [71]:
result[0]

{'Interesting': 2,
 'Engaging': 1,
 'Understandable': 3,
 'Relevant': 3,
 'Correct': 2,
 'Appropriate': 3,
 'Fluently': 3,
 'Direct': 2}

In [74]:
# result[0]
agg = {
'Interesting': 0,'Engaging': 0,'Understandable': 0,
 'Relevant': 0,
 'Correct': 0,
 'Appropriate': 0,
 'Fluently': 0,
 'Direct': 0}
for i in range(1000):
    # print(o)
    o = result[i]
    # print(o)
    try:
        for key in agg:
            agg[key] += o[key]
    except:
        for key in agg:
            agg[key] += 1
    # break

In [78]:
with open("gpt_score_llama3.json","w") as f:
    json.dump(result, f,indent=4)